# Лабораторная работа 1
Выполнил студент группы 0385 Иванов Серафим

In [9]:
import xml.etree.ElementTree as ET
from collections import defaultdict
import urllib.request
from tqdm import tqdm
import bz2
import os
import pickle
import re
import shutil
import zipfile

## Скачивание нужных файлов

In [75]:
import os

# Папка для сохранения файлов
output_dir = 'downloads'
os.makedirs(output_dir, exist_ok=True)

# Ссылки на файлы
urls = {
    'opencorpora_dict.xml.bz2': 'https://opencorpora.org/files/export/dict/dict.opcorpora.xml.bz2',
    'сrime_and_punishment.txt': 'https://bookex.info/uploads/public_files/2023-02/prestuplenie-i-nakazanie_-fedor-dostoevskij.txt',
    'war_and_peace.txt': 'https://gist.github.com/Semionn/bdcb66640cc070450817686f6c818897/raw/f9e8c888a771dd96f54562a9b050acd1138cc7a9/war_and_peace.ru.txt'
}

# Скачивание файлов
for filename, url in urls.items():
    output_path = os.path.join(output_dir, filename)
    urllib.request.urlretrieve(url, output_path)
    print(f'Файл "{filename}" успешно скачан в "{output_path}"')


Файл "opencorpora_dict.xml.bz2" успешно скачан в "downloads\opencorpora_dict.xml.bz2"
Файл "сrime_and_punishment.txt" успешно скачан в "downloads\сrime_and_punishment.txt"
Файл "war_and_peace.txt" успешно скачан в "downloads\war_and_peace.txt"


In [76]:
# Перемещаем все файлы из downloads в родительскую папку
parent_dir = os.path.dirname(output_dir)

for filename in os.listdir(output_dir):
    src_path = os.path.join(output_dir, filename)
    dst_path = os.path.join(parent_dir, filename)
    if os.path.isfile(src_path):
        shutil.move(src_path, dst_path)
        print(f'file "{filename}" mod=vet into "{dst_path}"')

file "opencorpora_dict.xml.bz2" mod=vet into "opencorpora_dict.xml.bz2"
file "war_and_peace.txt" mod=vet into "war_and_peace.txt"
file "сrime_and_punishment.txt" mod=vet into "сrime_and_punishment.txt"


## Обработка словаря

In [77]:
BZ2_PATH = "dict.opcorpora.xml.bz2"   # сжатый словарь
XML_PATH = "dict.opcorpora.xml"       # распакованный xml
PICKLE_PATH = "lemma_dict.pkl"        # куда сохраняем готовый словарь

if not os.path.exists(XML_PATH) and os.path.exists(BZ2_PATH):
    print("Unpacking dict...")
    with bz2.open(BZ2_PATH, "rb") as f_in, open(XML_PATH, "wb") as f_out:
        f_out.write(f_in.read())
    print("Done.")

In [78]:
GRAMM_TO_SIMPLE = {
    "NOUN": "S", "S": "S", "NPRO": "NI",
    "ADJF": "A", "ADJS": "A", "COMP": "A",
    "VERB": "V", "INFN": "V", "V": "V",
    "PRTF": "V", "PRTS": "V", "GRND": "V",
    "ADVB": "ADV", "ADV": "ADV", "NUMR": "NUM", "NUM": "NUM",
    "PREP": "PR", "PR": "PR", "CONJ": "CONJ", "PRCL": "PART", "INTJ": "INTJ",
}
POS_GRAMMEMES = set(GRAMM_TO_SIMPLE.keys())

In [79]:
def normalize(word: str) -> str:
    """Lower case + ё->е"""
    return word.lower().replace("ё", "е")

In [80]:
print("Counting <lemma>...")
lemma_count = 0
for _, elem in ET.iterparse(XML_PATH, events=("end",)):
    if elem.tag == "lemma":
        lemma_count += 1
    elem.clear()
print(f"Total: {lemma_count:,}")

Counting <lemma>...
Total: 391,845


In [81]:
lemma_dict = dict()

print("Creating lemma dict...")
it = ET.iterparse(XML_PATH, events=("end",))
with tqdm(total=lemma_count, unit="lemma") as pbar:
    for event, elem in it:
        if elem.tag == "lemma":
            l_elem = elem.find("l")
            if l_elem is not None:
                lemma_text = l_elem.get("t") or ""
                pos_simple = None
                for g in l_elem.findall("g"):
                    v = g.get("v")
                    if v in POS_GRAMMEMES:
                        pos_simple = GRAMM_TO_SIMPLE[v]
                        break
                if pos_simple is None:
                    pos_simple = "?"

                # обавляем все формы <f> плюс лемму ---
                forms = [lemma_text] + [f.get("t") for f in elem.findall("f") if f.get("t")]
                for form in forms:
                    norm = normalize(form)
                    lemma_norm = normalize(lemma_text)
                    lemma_dict[norm] = (lemma_norm, pos_simple)

            elem.clear()
            pbar.update(1)

print("The dict is ready, total keys:", len(lemma_dict))

Creating lemma dict...


100%|██████████| 391845/391845 [00:44<00:00, 8813.50lemma/s] 

The dict is ready, total keys: 3060604


In [82]:
with open(PICKLE_PATH, "wb") as f:
    pickle.dump(lemma_dict, f, protocol=4)
print("Saved to", PICKLE_PATH)

Saved to lemma_dict.pkl


## Обработка текста

Загружаем готовый словарь

In [10]:
PICKLE_PATH = "lemma_dict.pkl"
with open(PICKLE_PATH, "rb") as f:
    lemma_dict = pickle.load(f)
print("Dict loaded, keys:", len(lemma_dict))

Dict loaded, keys: 3060604


Функции для лемматизации

In [12]:
# глобальные счётчики
total_tokens = 0
not_found_tokens = 0

def normalize(word: str) -> str:
    """Normalize a token: convert to lowercase and replace 'ё' with 'е'"""
    return word.lower().replace("ё", "е")

def lemmatize_token(token: str):
    """
    Returns the token in the format: token{lemma=POS}.
    Updates global counters:
      - total_tokens
      - not_found_tokens
    """
    global total_tokens, not_found_tokens
    total_tokens += 1

    norm = normalize(token)
    if norm in lemma_dict:
        lemma, pos = lemma_dict[norm]
        return f"{token}{{{lemma}={pos}}}"
    else:
        not_found_tokens += 1
        return f"{token}{{{norm}=??}}"

def process_sentence(sentence: str):
    """
    1. Remove all punctuation
    2. Split the sentence into tokens
    3. Lemmatize each token (updates counters)
    4. Join tokens back into a string
    """
    clean = re.sub(r"[^\w\s]", "", sentence, flags=re.UNICODE)
    tokens = clean.split()
    return " ".join(lemmatize_token(tok) for tok in tokens)


Использование

In [13]:
# Сброс глобальных счётчиков перед обработкой
total_tokens = 0
not_found_tokens = 0

input_text = """Стала стабильнее экономическая и политическая обстановка, предприятия вывели из тени зарплаты сотрудников.
Все Гришины одноклассники уже побывали за границей, он был чуть ли не единственным, кого не вывозили никуда дальше Красной Пахры.
"""

print("Input:")
print(input_text)

print("\nOutput:")
for line in input_text.strip().split("\n"):
    print(process_sentence(line))

# Печатаем статистику
print("\nStatistics:")
print(f"Total tokens processed: {total_tokens}")
print(f"Tokens not found in dictionary: {not_found_tokens} ({not_found_tokens/total_tokens:.2%})")

Input:
Стала стабильнее экономическая и политическая обстановка, предприятия вывели из тени зарплаты сотрудников.
Все Гришины одноклассники уже побывали за границей, он был чуть ли не единственным, кого не вывозили никуда дальше Красной Пахры.


Output:
Стала{стал=V} стабильнее{стабильнее=A} экономическая{экономический=A} и{и=S} политическая{политический=A} обстановка{обстановка=S} предприятия{предприятие=S} вывели{вывел=V} из{иза=S} тени{тень=S} зарплаты{зарплата=S} сотрудников{сотрудник=S}
Все{все=PART} Гришины{гришин=A} одноклассники{одноклассник=S} уже{уже=PART} побывали{побывал=V} за{за=PR} границей{граница=S} он{он=NI} был{есть=V} чуть{чуть=CONJ} ли{ли=S} не{не=PART} единственным{единственный=A} кого{кто=NI} не{не=PART} вывозили{вывозил=V} никуда{никуда=ADV} дальше{дальше=A} Красной{красный=A} Пахры{пахра=S}

Statistics:
Total tokens processed: 32
Tokens not found in dictionary: 0 (0.00%)


## Обработка большого текста

In [6]:
INPUT_FILE = "сrime_and_punishment.txt"   # путь к файлу с текстом
OUTPUT_FILE = "сrime_and_punishment_out.txt" # куда сохранить результат

In [7]:
# Кодировки:
# Война и мир UTF-8
# Преступление и наказание windows 1251

# Сброс глобальных счётчиков перед обработкой
total_tokens = 0
not_found_tokens = 0

with open(INPUT_FILE, "r", encoding="UTF-8") as f:
    lines = f.readlines()

processed_lines = []
for line in lines:
    line = line.strip()
    if line:  # skip empty lines
        processed_lines.append(process_sentence(line))

In [8]:
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    for line in processed_lines:
        f.write(line + "\n")

print("Processed text:")
for line in processed_lines[2000:2010]:
    print(line)

# Печатаем статистику
print("\nStatistics:")
print(f"Total tokens processed: {total_tokens}")
print(f"Tokens not found in dictionary: {not_found_tokens} ({not_found_tokens/total_tokens:.2%})")

Processed text:
Она{она=NI} поминки{поминки=S} устраивает{устраиваю=V}
Дас{дас=??} закуску{закуска=S} она{она=NI} вас{вы=NI} очень{очень=ADV} велела{велю=V} благодарить{благодарить=V} что{что=PART} вы{вы=NI} вчера{вчера=ADV} помогли{помог=V} нам{мы=NI} без{без=PR} вас{вы=NI} совсем{совсем=ADV} бы{бы=PART} нечем{нечего=NI} похоронить{похоронить=V} И{и=S} губы{губа=S} и{и=S} подбородок{подбородок=S} ее{она=NI} вдруг{вдруг=ADV} запрыгали{запрыгал=V} но{но=INTJ} она{она=NI} скрепилась{скрепился=V} и{и=S} удержалась{удержался=V} поскорей{скорее=A} опять{опять=ADV} опустив{опустив=V} глаза{глаз=S} в{в=S} землю{земля=S}
Между{между=PR} разговором{разговор=S} Раскольников{раскольников=S} пристально{пристален=A} ее{она=NI} разглядывал{разглядываю=V} Это{этот=A} было{есть=V} худенькое{худенький=A} совсем{совсем=ADV} худенькое{худенький=A} и{и=S} бледное{бледный=A} личико{личико=S} довольно{доволен=A} неправильное{неправильный=A} какоето{какоето=??} востренькое{востренький=A} с{с=PART} востреньки